# Data Fusion Contest 2026 - Задача 3 "Герои"

In [1]:
import pandas as pd
import numpy as np

In [2]:
h_df = pd.read_csv('/kaggle/input/data-fusion-2026-case-3/data_heroes.csv')
o_df = pd.read_csv('/kaggle/input/data-fusion-2026-case-3/data_objects.csv')
d_obj = pd.read_csv('/kaggle/input/data-fusion-2026-case-3/dist_objects.csv')
d_start = pd.read_csv('/kaggle/input/data-fusion-2026-case-3/dist_start.csv')

In [3]:
dist_m = d_obj.values
start_dists = d_start.set_index('object_id')['dist_start'].to_dict()
objs = o_df.to_dict('records')
heroes = h_df.to_dict('records')

In [4]:
v_cost = 100
h_price = 2500
max_days = 7

In [5]:
used = set()
res = []
total_g = 0
max_h_idx = -1

# основной цикл решения 

In [6]:
for i, h in enumerate(heroes):
    h_id = h['hero_id']
    m_pts = h['move_points']

    curr_obj = None
    curr_day = 1
    curr_pts = m_pts
    h_route = []
    h_gold = 0

    while True:
        best_o = None
        min_extra = float('inf')

        for o in objs:
            idx = o['object_id']
            if idx in used: continue

            d = start_dists[idx] if curr_obj is None else dist_m[curr_obj - 1][idx - 1]

            t_day = curr_day
            t_pts = curr_pts

            needed = d
            while needed > t_pts:
                needed -= t_pts
                t_day += 1
                t_pts = m_pts
            t_pts -= needed

            if t_day > o['day_open']: continue

            if t_day < o['day_open']:
                t_day = o['day_open']
                t_pts = m_pts

            if t_pts < v_cost:
                if t_pts > 0:
                    t_pts = 0
                else:
                    t_day += 1
                    t_pts = m_pts - v_cost
            else:
                t_pts -= v_cost

            if t_day <= max_days:
                score = (t_day * 10000) + d
                if score < min_extra:
                    min_extra = score
                    best_o = (o, t_day, t_pts)

        if best_o:
            o_data, curr_day, curr_pts = best_o
            oid = o_data['object_id']
            h_route.append(oid)
            used.add(oid)
            curr_obj = oid
            h_gold += 500
        else:
            break

    if h_gold > h_price or (not res and h_gold > 0):
        for oid in h_route:
            res.append([h_id, oid])
        total_g += h_gold
        max_h_idx = i
    elif i > max_h_idx + 5:
        break

In [7]:
out = pd.DataFrame(res, columns=['hero_id', 'object_id'])
out.to_csv('ggg.csv', index=False)